In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Installation


In [ ]:
!pip install -U --pre tensorflow=="2.2.0"

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
!apt-get install protobuf-compiler python-lxml python-pil
!pip install Cython pandas tf-slim

In [ ]:
%cd models/research

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
import os
os.environ['PYTHONPATH'] += ':/content/models/research/slim/'
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/object_detection/'
os.environ['PYTHONPATH'] += ':/content/models'

In [ ]:
!cp object_detection/packages/tf2/setup.py .

In [ ]:
!python -m pip install .

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
%cd /content/

# Downloading Pre-Trained Model


In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz

In [ ]:
!tar -xvf  /content/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz

In [ ]:
%cd /content/models/research/object_detection/

# Include Tensorboard

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir "/train_log/"

# Training

In [ ]:
!python model_main_tf2.py --model_dir="/train_log" --pipeline_config_path=/content/ssd_mobilenet_v2_320x320_coco17_tpu-8/pipeline.config

# Evaluating


In [ ]:
!python model_main_tf2.py --model_dir="/train_log" --pipeline_config_path=/content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config --checkpoint_dir="/train_log"

# Exporting to Converting TFLite 

In [ ]:
!python export_tflite_graph_tf2.py --pipeline_config_path=/content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config --trained_checkpoint_dir="/train_log" --output_directory="/export_lite"

In [ ]:
!pip install tf-nightly

In [ ]:

import tensorflow as tf
import numpy as np

#uncomment blow to create  representative dataset
"""
filenames = ["/data/train.record"]
image_feature_description = {'image/encoded': tf.io.FixedLenFeature([], tf.string)}
raw_dataset = tf.data.TFRecordDataset(filenames)

def _parse_image_function(example_proto):
  return tf.io.parse_single_example(example_proto, image_feature_description)

def representative_data_gen():
    
  parsed_image_dataset = raw_dataset.map(_parse_image_function)
  

  for image_features in parsed_image_dataset:
    #image_raw = image_features['image/encoded'].numpy()
    #image = tf.io.decode_raw(image_raw,out_type=tf.float32)
    image = tf.image.decode_image(image_features['image/encoded'],dtype=tf.float32)
    image = tf.image.resize(image, [320, 320])
    #image = tf.cast(image / 255., tf.float32)
    image = np.asarray(image)
    image = np.expand_dims(image,0)
    
    yield [image]

"""


saved_model_dir = "/export_lite/saved_model"
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

converter.optimizations = [tf.lite.Optimize.DEFAULT]

#uncomment below to make change
"""
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8,tf.lite.OpsSet.TFLITE_BUILTINS]
converter.representative_dataset = representative_data_gen

converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
converter.target_spec.supported_types = [tf.int8]
"""
tf_lite_model = converter.convert()
with open('/export_lite/detect_model.tflite', 'wb') as f:
  f.write(tf_lite_model)


**Alternative2**

In [ ]:
saved_model_dir = "/export_lite/saved_model"
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

tf_lite_model = converter.convert()
with open('/export_lite/detect_model.tflite', 'wb') as f:
  f.write(tf_lite_model)

**Alternative3**

In [ ]:
!tflite_convert --saved_model_dir=/export_lite/saved_model --output_file=/export_lite/detect_model.tflite